# Huc 12 Predictors
## Test 19020302 HUC 8 incl chugach nforest
All Huc 12s- Intersect w/catchments and find catchment (find downstream most catchment) with most upstream contribution and build watersheds for all ds catchments

Keep -
mnwtd_slope
% lake
summer precip - Runs in R but need downstream point (centroid of ds catchment)
predict for huc8

Visualize by HUC12 - Join data back to Huc12

## Section
Import modules
Set/Create Workspaces
 * Create Huc12 Catchment dataset to store catchments that will have watersheds generated
 * Crate Catchment point dataset to store catchment centroids
Collect Data
 * Set up data dictionary with VPU as key and store path to source data
    * Slope - Can link to already created slope rasters from original covariates worflow
    * NHDPlus Waterbodies - Can link to waterbody feature clase from original covariate workflow
Merge Data
 * Merge data together and or copy to local/in_memory and check that all projections match



In [41]:
import os, arcpy, sys
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

print('imports complete')
print(f'sys paths {sys.path}')
print (datetime.datetime.now())


imports complete
sys paths ['C:\\Program Files\\JetBrains\\PyCharm 2020.2.3\\plugins\\python\\helpers-pro\\jupyter_debug', 'C:\\Program Files\\JetBrains\\PyCharm 2020.2.3\\plugins\\python\\helpers\\pydev', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\data_preparation\\sensitivity_drivers\\geomorphology', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\Resources\\ArcPy', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python37.zip', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\DLLs', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3', '', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages', 'C:\\Users\\dwmer

In [42]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value

## Collect NHDPlus Datasets

In [43]:
#Create Dictionary to relate datasets to vpu
vpuDict = {}
# Walk through folders and subfolders to collect vpus

#Path to folder with NHDPlus data separated by vpu
nhdPlusfol = r'D:\Basedata\NHDPlus'
rois = ['Cook_Inlet','Copper_River']
vpuList = []

#NHD folders - Update to new data downloaded Nov 8, 2021
for roi in rois:
    arcpy.env.workspace = os.path.join(nhdPlusfol,roi)
    gdbs = arcpy.ListWorkspaces('NHDPLUS*','FileGDB')
    print(gdbs)
    Cats = []
    VAA = []
    waterbodies = []
    flowlines = []

    for gdb in gdbs:
        arcpy.env.workspace = gdb
        huc = gdb[-20:-12]
        append_value(vpuDict,huc,gdb)
        print(huc)
        vpuList.append(huc)
        datasets = arcpy.ListDatasets(feature_type='feature')
        for ds in datasets:
            for fc in arcpy.ListFeatureClasses(feature_dataset=ds):
                if fc == "NHDFlowline":
                    flowpath = os.path.join(arcpy.env.workspace, ds, fc)
                    flowname = 'flowline_' + str(huc)
                    append_value(vpuDict,huc,flowpath)
                #     print(f'Copying {flowname}...')
                #     flowcopypath = os.path.join(scratch, flowname)
                #     arcpy.management.CopyFeatures(flowpath, flowcopypath)
                #     flowlines.append(flowcopypath)
                elif fc == "NHDWaterbody":
                    waterbodyname = 'waterbody_' + str(huc)
                    #waterbodycopypath = os.path.join(scratch, waterbodyname)
                    waterpath = os.path.join(arcpy.env.workspace, ds, fc)
                    #print(f'Copying {waterbodyname}...')
                    append_value(vpuDict,huc,waterpath)
                    # arcpy.management.CopyFeatures(waterpath, waterbodycopypath)
                    # waterbodies.append(waterbodycopypath)
                elif fc == "NHDPlusCatchment":
                    catchpath = os.path.join(arcpy.env.workspace, ds, fc)
                    append_value(vpuDict, huc, catchpath)
                elif fc == "WBDHU12":
                    hucpath = os.path.join(arcpy.env.workspace, ds, fc)
                    append_value(vpuDict, huc, hucpath)

        vaapath = os.path.join(arcpy.env.workspace, "NHDPlusFlowlineVAA")
        #print(f'Vaa table at {vaapath} exists = {arcpy.Exists(vaapath)}')
        append_value(vpuDict,huc,vaapath)



['D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020202_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020301_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020302_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020401_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020402_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020501_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020502_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020503_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020504_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020505_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020601_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020602_HU8_GDB.gdb', 'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020800_HU8_GDB.gdb']
19020202
19020301
19020302
19020401
19020402
19020501
19020502
19020503
19020504
19020505
19020601
190

In [44]:
# Set data dir equal to directory containing the AKSSF regional sub-folders.
akssf_data_dir = r'D:\\GIS\\AKSSF'
nhd_data_dir = r'D:\\Basedata\\NHDPlus'

# Create dictionaries
nhdDict = {}
tauDict = {}
inputDict = {"'":"",'"':""}
scrDict = {}

def replace_all(userinput, dic):
    for i, j in dic.items():
        userinput = userinput.replace(i, j)
    return userinput


# Create scratch workspace
while True:
    try:
        userinput = replace_all((input('Input drive or directory to create scratch workspaces ex. \'W:\\\GIS\'\n') or 'D:\\GIS'),inputDict)
        if not arcpy.Exists(userinput):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            scratchdrive = userinput
            break
    except arcpy.ExecuteError:
            print(arcpy.GetMessages())

huc12cv_scratchgdb = os.path.join(scratchdrive,r"\\huc12_Outputs\\huc12_scratch.gdb")
huc12cv_scratchfol = os.path.dirname(huc12cv_scratchgdb)
print(f'Scratch folder will be created {huc12cv_scratchfol}\nScratch GDB at {huc12cv_scratchgdb} ')

# Specify path to AWC events fc
while True:
    try:
        userinput2 = replace_all((input('Input path to awc events feature class or shapefile. \'"J:\\GIS_data\\biota\\Aquatic\\Fauna\\AWC\\2021_Species_LifeStage.gdb\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs"\'\n') or
                                  'D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs'), inputDict)
        if not arcpy.Exists(userinput2):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            awc_events = userinput2
            break
    except arcpy.ExecuteError:
            print(arcpy.GetMessages())

print(f'Using AWC events from {awc_events}\n {"-"*100}')

# Create Scratch Workspaces and add to dictionary
if not arcpy.Exists(huc12cv_scratchfol):
    os.mkdir(huc12cv_scratchfol)
    print(f'Creating scratch folder {huc12cv_scratchfol}')
else:
    print(f'Scratch folder {huc12cv_scratchfol} already exists')
print('----------')

if not arcpy.Exists(huc12cv_scratchgdb):
    arcpy.CreateFileGDB_management(huc12cv_scratchfol,"huc12_scratch.gdb")
    print(f'Creating scratch gdb {huc12cv_scratchgdb}')
else:
    print(f'Scratch folder {huc12cv_scratchgdb} already exists')
print('----------')

# Create VPU output folders
for vpu in vpuList:
    vpu_name = os.path.basename(vpu)
    hucscratchpath = os.path.join(huc12cv_scratchfol, vpu_name + '_h12')
    append_value(scrDict,vpu_name,hucscratchpath)
    if not arcpy.Exists(hucscratchpath):
        os.mkdir(hucscratchpath)
        print (f'{vpu_name} does not have a scratch folder')
        print (f'Creating contributing area scratch folder at  {hucscratchpath}')
    else:
        print(f'Scratch folder for {vpu_name} already created at {hucscratchpath}')

    print('----------')
print(f'All scratch workspaces set')
scrDict


Scratch folder will be created D:\\huc12_Outputs
Scratch GDB at D:\\huc12_Outputs\\huc12_scratch.gdb 
Using AWC events from D:\Basedata\AWC\AWC_2021_SpeciesEvents.gdb\awcEventArcs
 ----------------------------------------------------------------------------------------------------
Scratch folder D:\\huc12_Outputs already exists
----------
Scratch folder D:\\huc12_Outputs\\huc12_scratch.gdb already exists
----------
Scratch folder for 19020202 already created at D:\\huc12_Outputs\19020202_h12
----------
Scratch folder for 19020301 already created at D:\\huc12_Outputs\19020301_h12
----------
Scratch folder for 19020302 already created at D:\\huc12_Outputs\19020302_h12
----------
Scratch folder for 19020401 already created at D:\\huc12_Outputs\19020401_h12
----------
Scratch folder for 19020402 already created at D:\\huc12_Outputs\19020402_h12
----------
Scratch folder for 19020501 already created at D:\\huc12_Outputs\19020501_h12
----------
Scratch folder for 19020502 already created at 

{'19020202': 'D:\\\\huc12_Outputs\\19020202_h12',
 '19020301': 'D:\\\\huc12_Outputs\\19020301_h12',
 '19020302': 'D:\\\\huc12_Outputs\\19020302_h12',
 '19020401': 'D:\\\\huc12_Outputs\\19020401_h12',
 '19020402': 'D:\\\\huc12_Outputs\\19020402_h12',
 '19020501': 'D:\\\\huc12_Outputs\\19020501_h12',
 '19020502': 'D:\\\\huc12_Outputs\\19020502_h12',
 '19020503': 'D:\\\\huc12_Outputs\\19020503_h12',
 '19020504': 'D:\\\\huc12_Outputs\\19020504_h12',
 '19020505': 'D:\\\\huc12_Outputs\\19020505_h12',
 '19020601': 'D:\\\\huc12_Outputs\\19020601_h12',
 '19020602': 'D:\\\\huc12_Outputs\\19020602_h12',
 '19020800': 'D:\\\\huc12_Outputs\\19020800_h12',
 '19020101': 'D:\\\\huc12_Outputs\\19020101_h12',
 '19020102': 'D:\\\\huc12_Outputs\\19020102_h12',
 '19020103': 'D:\\\\huc12_Outputs\\19020103_h12',
 '19020104': 'D:\\\\huc12_Outputs\\19020104_h12'}

## Section 2
### By Vector Processing Unit
Identify downstream-most catchment for each Huc 12
 * Select by location and select catchment with most us contributing area
    * NHDPlus
        * Use update cursor to join TotalDrainageAreaSqKm from vaa table to catchment
        * Find max value from selection and save as outlet catchment for that HUC12
    * TauDEM
        * DSContArea - Drainage area at the downstream end of the link. Generally this is one grid cell upstream of the downstream end because the drainage area at the downstream end grid cell includes the area of the stream being joined.
 * Generate Centroid point and append to centroid dataset
    * Retain cat_id and Huc12-id
 * Append to HUC12 catchment dataset


In [45]:
c=0
for v in vpuDict['19020302']:
    print(f'{v}. {c}')
    c+=1

D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb. 0
D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\NHDPlus\NHDPlusCatchment. 1
D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\WBD\WBDHU12. 2
D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\Hydrography\NHDFlowline. 3
D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\Hydrography\NHDWaterbody. 4
D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\NHDPlusFlowlineVAA. 5


In [48]:
# List of HUC12s to be processed
import operator
proclist = ['19020302']
outletDict = {}
outletList = []
outletcats = []
outletcatpts = []

for vpu in vpuList:
    if vpu in proclist:
        try:
            print(f'Huc {vpu} will be processed')
            # Inputs
            cats = vpuDict[vpu][1]
            vaas = vpuDict[vpu][5]
            hucs = vpuDict[vpu][2]
            lakes = vpuDict[vpu][4]
            #Output names and paths
            outletcatsname = 'HUC_' + str(vpu) +'_out_cats'
            outcatspath = os.path.join(huc12cv_scratchgdb,outletcatsname)
            outletcatptsname = 'HUC_' + str(vpu) +'_out_catspts'
            outcatptspath = os.path.join(huc12cv_scratchgdb,outletcatptsname)
            # Select by expression for outlet cats

            print (f'Catchments located {cats}')
            print (f'Vaas located {vaas}')
            print (f'NHD waterbodies located {lakes}')
            print (f'Huc12s located {hucs}')
            # Use list comprehension to build a dictionary from a da SearchCursor
            fields = ['NHDPlusID','TotDASqKm']
            # Build Value dictionary to relate NHDPlus id to contributing area
            valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(vaas, fields)}
            hucselect = arcpy.SelectLayerByLocation_management(hucs,'INTERSECT',awc_events,'','NEW_SELECTION')
            print(('*'*100))
            print(f'{arcpy.GetCount_management(hucselect)} of {arcpy.GetCount_management(hucs)} HUC12s in {vpu} intersect awc events input')
            with arcpy.da.SearchCursor(hucselect,['HUC12','SHAPE@']) as cur:
                for row in cur:
                    #print(f'Processing HUC {row[0]}')
                    inhuc = row[1]
                    cat_layer = arcpy.MakeFeatureLayer_management(cats,'cat_layer')
                    # Select by location using awc and huc 12
                    arcpy.SelectLayerByLocation_management(cat_layer,'HAVE_THEIR_CENTER_IN',inhuc,'','NEW_SELECTION')
                    print(f'Finding outlet for HUC {row[0]} out of {arcpy.GetCount_management(cat_layer)} catchments.\n{("*" * 60)}')
                    catList = [r[0] for r in arcpy.da.SearchCursor(cat_layer, 'NHDPlusID')]
                    intersect = list(set(catList).intersection(valueDict))
                    catDict = {i:(valueDict[i]) for i in intersect}
                    # Find Catchment with max drainage area
                    outcatch = max(catDict.items(), key = operator.itemgetter(1))[0]
                    append_value(outletDict, row[0], int(outcatch))
                    outletList.append(int(outcatch))
            outlet_cats = arcpy.MakeFeatureLayer_management(cats,'outlet_cats')
            out_expression ='"NHDPlusID" IN ' + str(tuple(outletList))
            #print(out_expression)
            outlet_cats_select = arcpy.SelectLayerByAttribute_management(outlet_cats,'NEW_SELECTION', out_expression)
            print(f'Creating copy of {arcpy.GetCount_management(outlet_cats)} outlet catchments for Huc {vpu} at {outcatspath}')
            print(('*'*100))
            arcpy.FeatureClassToFeatureClass_conversion(outlet_cats_select,huc12cv_scratchgdb,outletcatsname)
            cat_points = arcpy.FeatureToPoint_management(outcatspath, outcatptspath, 'INSIDE')
            outletcats.append(outcatspath)
            outletcatpts.append(outcatptspath)

        except Exception:
            e = sys.exc_info()[1]
            print(e.args[0])
            arcpy.AddError(e.args[0])

    else:
        print(('-'*100))
        print(f'Huc {vpu} will not be processed')
        print(('-'*100),'\n')





----------------------------------------------------------------------------------------------------
Huc 19020202 will not be processed
---------------------------------------------------------------------------------------------------- 

----------------------------------------------------------------------------------------------------
Huc 19020301 will not be processed
---------------------------------------------------------------------------------------------------- 

Huc 19020302 will be processed
Catchments located D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\NHDPlus\NHDPlusCatchment
Vaas located D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\NHDPlusFlowlineVAA
NHD waterbodies located D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\Hydrography\NHDWaterbody
Huc12s located D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\WBD\WBDHU12
*******************************************************************************************

## Section 3
### Generate watersheds
Iterate over HUC12 catchment dataset and create watersheds
 * Append output wtd to wtd dataset - Link back to catchment using catID (create catIDcon for unique identifier field)

## Section 4
###Calculate covariates
Iterate over watersheds and calcate the following:
* Mean wtd slope
* % Lake Cover
* Mean LCLD
* % Glacier Cover
